In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pickle

In [12]:
important_features = ['Credit_Score', 'Age_Oldest_TL', 'enq_L3m', 'enq_L6m', 'num_std', 'time_since_recent_enq', 'num_std_12mts', 'pct_PL_enq_L6m_of_L12m', 'num_std_6mts', 'time_since_recent_deliquency','Approved_Flag']

In [13]:
df1 = pd.read_excel("C:/Users/Admin/Python Techpaathshala/Credit_Modelling/case_study1.xlsx")
df2 = pd.read_excel("C:/Users/Admin/Python Techpaathshala/Credit_Modelling/case_study2.xlsx")
df = pd.merge(df1, df2, on='PROSPECTID', how='inner')
df = df.drop(columns=['PROSPECTID'])
#cols_to_drop = [col for col in df.columns if (df[col] == -99999).sum() > 10000]
#df_cleaned = df.drop(columns=cols_to_drop)
#for col in df_cleaned.columns:
#    df_cleaned = df_cleaned[df_cleaned[col] != -99999]

df_cleaned = df[important_features]

print(df_cleaned)

       Credit_Score  Age_Oldest_TL  enq_L3m  enq_L6m  num_std  \
0               696             72        0        0       21   
1               685              7        0        0        0   
2               693             47        0        0       10   
3               673              5   -99999   -99999        5   
4               753            131        0        0       53   
...             ...            ...      ...      ...      ...   
51331           650             24        1        3        0   
51332           702             74        0        0        6   
51333           661              9        2        4        0   
51334           686             15        0        0        0   
51335           681             20        1        1       18   

       time_since_recent_enq  num_std_12mts  pct_PL_enq_L6m_of_L12m  \
0                        566             11                     0.0   
1                        209              0                     0.0   
2     

In [4]:
# Extract feature names from the preprocessor
rf_preprocessor = rf_pipeline.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()

# Extract feature importances from the classifier
rf_importances = rf_pipeline.named_steps['classifier'].feature_importances_

# Create and sort the importance DataFrame
rf_feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
rf_feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Only look at the Top 15
print(rf_feature_importance_df.head(15))

NameError: name 'rf_pipeline' is not defined

In [ ]:
# Extract feature names from the preprocessor
gb_preprocessor = gb_pipeline.named_steps['preprocessor']

# Extract feature importances from the classifier
gb_importances = gb_pipeline.named_steps['classifier'].feature_importances_

# Create and sort the importance DataFrame
gb_feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
gb_feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Only look at the Top 15
print(gb_feature_importance_df.head(15))
important_features = []

for i in gb_feature_importance_df.head(15)['Feature']:
    important_features.append(i)
    
    
print(important_features)

In [16]:
# 4. Feature Selection
target = 'Approved_Flag'
X = df_cleaned.drop(columns=[target])
y = df_cleaned[target]

# Label encode the target variable
le = LabelEncoder()
y = le.fit_transform(y)

# Identify numerical and categorical columns for the pipeline
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

# 5. Define Preprocessing Pipeline
num_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

cat_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('label', LabelEncoder(handle_unknown='ignore'))
    ])

preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_cols),
            ('cat', cat_transformer, cat_cols)
        ]
    )

    # 6. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # 7. Model Training - Random Forest Pipeline
rf_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])
print("Training Random Forest...")
rf_pipeline.fit(X_train, y_train)

    # 8. Model Training - Gradient Boosting Pipeline
gb_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier(n_estimators=100, random_state=42))
    ])
print("Training Gradient Boosting...")
gb_pipeline.fit(X_train, y_train)

TypeError: LabelEncoder() takes no arguments

In [6]:
y_pred_gb = gb_pipeline.predict(X_test)
print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))
y_pred_rf = rf_pipeline.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Gradient Boosting Accuracy: 0.9952278924814959
Random Forest Accuracy: 0.9887027658745617


In [10]:

# 9. Save Models and Label Encoder as Pickle Files
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(rf_pipeline, f)

with open('gradient_boosting_model.pkl', 'wb') as f:
    pickle.dump(gb_pipeline, f)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

print("Training complete. Models saved successfully.")

Training complete. Models saved successfully.
